In [12]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [6]:
dataset_path = 'data.json'

In [8]:
with open(dataset_path, 'r') as fp:
    data = json.load(fp)

In [9]:
inputs = np.array(data['mfcc'])
targets = np.array(data['labels'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.3)

In [17]:
inputs.shape

(4993, 259, 13)

In [19]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),
    
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    
    keras.layers.Dense(10, activation='softmax')
])

In [22]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
             loss='sparse_catagorical_crossentropy',
             metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3367)              0         
                                                                 
 dense (Dense)               (None, 512)               1724416   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,872,842
Trainable params: 1,872,842
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=50,
          batch_size=32)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\vivek\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "C:\Users\vivek\anaconda3\lib\site